# TRAINER
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512), 21.7.15)

PyTorch 코드를 LightningModule로 구성하면 Trainer가 다른 모든 것을 자동화합니다.

[가이드 동영상](https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/pl_docs/pt_trainer_mov.m4v)을 참고하세요!


Lightning은 코드를 추상화함으로써 다음 사항들을 달성할 수 있습니다.
- 추상화를 추가하지 않고도 PyTorch 코드를 통해 모든 측면을 제어할 수 있습니다.
- Trainer는 Facebook AI Research, NYU, MIT, Stanford 등과 같은 최고의 AI 연구소의 기여자 및 사용자가 내장한 모범 사례를 사용합니다.
- Trainer는 자동화를 원하지 않는 모든 핵심 부분을 재정의할 수 있습니다.

---


## 기본 사용법

Trainer의 기본적인 사용법은 다음과 같습니다.

In [ ]:
model = MyLightningModule()

trainer = Trainer()
trainer.fit(model, train_dataloader, val_dataloader)

---

## 내부 살펴보기

내부에서 Lightning Trainer는 훈련 루프 세부적인 정보를 처리합니다. 몇 가지 예는 다음과 같습니다.

- 자동으로 grads 활성화/비활성화
- 훈련, 검증 및 테스트 데이터로더 실행
- 적절한 시간에 콜백 호출
- 올바른 장치에 배치 및 연산 넣기

다음 예는 내부에서 Train loop가 돌아가는 과정을 묘사한 Psuedocode입니다.

In [ ]:
# put model in train mode
model.train()
torch.set_grad_enabled(True)

losses = []
for batch in train_dataloader:
    # calls hooks like this one
    on_train_batch_start()

    # train step
    loss = training_step(batch)

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

    losses.append(loss)

---

## 파이썬 스크립트 내의 Trainer

Python 스크립트에서는 기본 함수를 사용하여 Trainer를 호출하는 것이 좋습니다.

In [ ]:
from argparse import ArgumentParser

def main(hparams):
    model = LightningModule()
    trainer = Trainer(gpus=hparams.gpus)
    trainer.fit(model)

if __name__ == '__main__':
    parser = ArgumentParser()
    parser.add_argument('--gpus', default=None)
    args = parser.parse_args()

    main(args)

다음과 같이 말이죠

```bash
python main.py --gpus 2
```

> <span style='color:blue'>NOTE:</span> 전문가 팁: 여러분은 모든 플래그를 수동으로 정의할 필요가 없습니다. Lightning에서 자동으로 추가할 수 있습니다.

In [ ]:
from argparse import ArgumentParser

def main(args):
    model = LightningModule()
    trainer = Trainer.from_argparse_args(args)
    trainer.fit(model)

if __name__ == '__main__':
    parser = ArgumentParser()
    parser = Trainer.add_argparse_args(parser)
    args = parser.parse_args()

    main(args)

당신은 이렇게도 사용할 수 있습니다.
```bash
python main.py --gpus 2 --max_steps 10 --limit_train_batches 10 --any_trainer_arg x
```

> <span style='color:blue'>NOTE:</span> 훈련 실행을 일찍 중지하려면 키보드에서 "Ctrl + C"를 누르세요. 트레이너는 KeyboardInterrupt를 포착하고 메모리 정리를 위한 가속기 콜백 `on_train_end` 실행을 포함하여 정상적인 종료를 시도합니다. 이러한 경우 트레이너 개체는 `interrupted` attribute도 `True`로 설정합니다. 예를 들어 컴퓨팅 리소스를 종료하는 콜백이 있는 경우 중단되지 않은 실행에 대해서만 종료 논리를 조건부로 실행할 수 있습니다.

---

## Validation

[`pytorch_lightning.trainer.trainer.Trainer.validate()`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.trainer.trainer.html#pytorch_lightning.trainer.trainer.Trainer.validate)를 사용하여 훈련 루프 외부에서 Valid set에 대해 Evaluation epoch를 수행할 수 있습니다. 이는 초기화 시 또는 이미 학습된 후에 모델에서 새 메트릭을 수집하려는 경우에 유용할 수 있습니다.

In [ ]:
trainer.validate(dataloaders=val_dataloaders)

---

## Reproducibility

완벽한 재현성을 보장하기 위해서는, 의사 난수 생성기에 대한 시드를 설정하고 `Trainer`에서 `deterministic` flag를 설정해야 합니다.

다음 예를 보시죠

In [ ]:
from pytorch_lightning import Trainer, seed_everything

seed_everything(42, workers=True)
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
model = Model()
trainer = Trainer(deterministic=True)

[`seed_everything()`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.utilities.seed.html#pytorch_lightning.utilities.seed.seed_everything)에서 `workers=True`를 설정하면 Lightning은 모든 데이터로더 workers와 `torch`, [`numpy`](https://numpy.org/doc/stable/reference/index.html#module-numpy) 및 stdlib [`random`](https://docs.python.org/3/library/random.html#module-random) 난수 생성기에 대한 프로세스에서 고유한 시드를 파생합니다. 켜져 있으면 예를 들어 Data augmentation는 작업자 간에 반복되지 않습니다.

---
